## INTRO



In [ ]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', None)

## READ_CSV

In [ ]:
df = pd.read_csv('atuacoes_2.csv', encoding='UTF-8', nrows=200) #, dtype=str, nrows=1000)

df['ANO-FIM'] = df['ANO-FIM'].fillna(2024)
df['FLAG-DEDICACAO-EXCLUSIVA'] = df['FLAG-DEDICACAO-EXCLUSIVA'].replace('NAO', False).replace('SIM', True)

df = df.astype({
    'NRO-ID-CNPQ': int,
    'CODIGO-INSTITUICAO': str,
    'NOME-INSTITUICAO': str,
    'ANO-INICIO': int,
    'ANO-FIM': int,
    'ENQUADRAMENTO-FUNCIONAL': str,
    'TIPO-DE-VINCULO': str,
    'OUTRAS-INFORMACOES': str,
    'CARGA-HORARIA-SEMANAL': float,
    'FLAG-DEDICACAO-EXCLUSIVA': bool
})

df = df.fillna(' ')
df2 = df.copy(deep=True)

#df

In [ ]:
# LISTA DE IES NACIONAIS
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')
df_uni_filtered = df_uni_filtered.sort_values(['NOME_DA_IES'], ascending=False)
df_uni_filtered

## PRE-PROCESSAMENTO

In [ ]:
def clean_names(df, column_name):

    # Remover '&'
    df[column_name] = df[column_name].replace('&AMP;', ' ', regex=True)

    # Remover todos os caracteres nao alfa-numericos, exceto whitespace
    df[column_name] = df[column_name].str.replace('[^\w\s]', ' ', regex=True)

    # Normalizar a string
    df[column_name] = df[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # Remover stopwords
    stop = ['DO', 'DA', 'DE', 'E']
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    # Strip whitespace
    df[column_name] = df[column_name].apply(lambda x: x.strip())

    return df

In [ ]:
# Lista de Curriculo Lattes

# Tratar os nomes da NOME-INSTITUICAO
df2 = df2.apply(lambda x: x.astype(str).str.upper())
df2 = clean_names(df2, 'NOME-INSTITUICAO')

# Dropar linhas com IES < 50
#df2 = df2[df2.groupby('NOME-INSTITUICAO')['NOME-INSTITUICAO'].transform('count').ge(50)]

# Split df: educacao & outros
df2_edu = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['LIVRE', 'COLABORADOR', 'PROFESSOR_VISITANTE'])].copy(deep=True)
df2_outros = df2.loc[df2['ENQUADRAMENTO-FUNCIONAL'].isin(['SERVIDOR_PUBLICO', 'CELETISTA', 'NAN', 'OUTRO'])].copy(deep=True)

df2_edu

In [ ]:
# Lista de IES
df_uni_filtered = df_uni_filtered.apply(lambda x: x.astype(str).str.upper())
df_uni_filtered = clean_names(df_uni_filtered, 'NOME_DA_IES')

# merge colunas NOME e SIGLA
df_uni_filtered['NOME_E_SIGLA'] = df_uni_filtered['SIGLA'] + ' ' + df_uni_filtered['NOME_DA_IES']
df_uni_filtered

In [ ]:
# Split lista de IES por inicio do nome
df_uni_filtered.sort_values(by=["NOME_DA_IES"], ascending=True)

df_uni_outros = df_uni_filtered.copy(deep=True)

df_uni_instituto = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUTO')].index, inplace=True)

df_uni_faculdades = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('FACULDADE')].index, inplace=True)

df_uni_escola = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('ESCOLA')].index, inplace=True)

df_uni_universidade = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('UNIVERSIDADE')].index, inplace=True)

df_uni_instituicao = df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')]
df_uni_outros.drop(df_uni_outros[df_uni_outros['NOME_DA_IES'].str.contains('INSTITUICAO')].index, inplace=True)

# List
# df_uni_outros, df_uni_instituto, df_uni_faculdades, df_uni_escola, df_uni_universidade, df_uni_instituicao

## NLP


### Cosine

In [75]:
def get_uni_list_df(nome_instituicao):
    if 'INSTITUTO' in nome_instituicao:
        return df_uni_instituto, tfidf_matrix_instituto
    elif 'FACULDADE' in nome_instituicao:
        return df_uni_faculdades, tfidf_matrix_faculdades
    elif 'ESCOLA' in nome_instituicao:
        return df_uni_escola, tfidf_matrix_escola
    elif 'UNIVERSIDADE' in nome_instituicao:
        return df_uni_universidade, tfidf_matrix_universidade
    elif 'INSTITUICAO' in nome_instituicao:
        return df_uni_instituicao, tfidf_matrix_instituicao

    return df_uni_outros, tfidf_matrix_outros


def iterate_sigla(nome_instituicao, df_for_search):

    matched_rows = []

    print(nome_instituicao)

    for index, row in df_for_search.iterrows():

        if row['SIGLA']:

            pattern = r'\b' + row['SIGLA'] + r'\b'
            if re.search(pattern, nome_instituicao):
                matched_rows.append(row['SIGLA'])
                print("Matched SIGLA: ", row['SIGLA'], " - ", row['NOME_DA_IES'])

            # dos diversos matches, qual mais faz sentido?
            #if len(matched_rows) > 1:

    return ",".join(str(element) for element in matched_rows)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

threshold = 0.8

# Vetorização TF-IDF
vectorizer = TfidfVectorizer().fit(df_uni_filtered['NOME_DA_IES'])
tfidf_matrix_universidades = vectorizer.transform(df_uni_filtered['NOME_DA_IES']) # geral
tfidf_matrix_instituto = vectorizer.transform(df_uni_instituto['NOME_DA_IES'])
tfidf_matrix_faculdades = vectorizer.transform(df_uni_faculdades['NOME_DA_IES'])
tfidf_matrix_escola = vectorizer.transform(df_uni_escola['NOME_DA_IES'])
tfidf_matrix_universidade = vectorizer.transform(df_uni_universidade['NOME_DA_IES'])
tfidf_matrix_instituicao = vectorizer.transform(df_uni_instituicao['NOME_DA_IES'])
tfidf_matrix_outros = vectorizer.transform(df_uni_outros['NOME_DA_IES'])

# Nova Coluna com Resultado
df2_edu['nome_corrigido'] = ''
df2_edu['sigla_detectado'] = ''


# PLN
for index, row in df2_edu.iterrows():

    nome_instituicao = row['NOME-INSTITUICAO']
    print(nome_instituicao)

    # Ver se contém a sigla
    sigla_detectado = iterate_sigla(nome_instituicao, df_uni_filtered)
    if sigla_detectado:
        df2_edu.at[index, 'sigla_detectado'] = sigla_detectado

    # Se nao contem a sigla, procurar por nome completo
    else:
        vector_comparacao = vectorizer.transform([nome_instituicao])

        uni_list_df, tfidf_matrix_df = get_uni_list_df(nome_instituicao)

        similaridade = cosine_similarity(vector_comparacao, tfidf_matrix_df)

        max_value = np.max(similaridade)
        max_index = np.argmax(similaridade)

        nome_corrigido = uni_list_df.iloc[max_index]['NOME_DA_IES']

        print("nome corrigido : ", nome_corrigido, max_value)

        if (max_value >= threshold):
            df2_edu.at[index, 'nome_corrigido'] = nome_corrigido
        else:
            df2_edu.at[index, 'nome_corrigido'] = ''

    print("==================")

In [ ]:
# oq foi alterado
pd.set_option('display.max_rows', 10)
print(df2_edu[['NOME-INSTITUICAO', 'nome_corrigido']].loc[df2_edu['NOME-INSTITUICAO'] != df2_edu['nome_corrigido']])
pd.set_option('display.max_rows', 6)